Instalar dependencias

In [ ]:
pip install simpy

# Este es el simlib.h necesario para correr código de SIMLIB

In [ ]:
# Crear simlib.h
%%writefile simlib.h
#ifndef SIMLIB_H
#define SIMLIB_H

#include <stdio.h>
#include <stdlib.h>
#include <math.h>

#define EVENT_LIST_SIZE 3
#define EVENT_TIME 1
#define EVENT_TYPE 2

#ifndef INFINITY
#define INFINITY 1.0e30
#endif


extern int next_event_type, maxatr;
extern double sim_time, event_list[EVENT_LIST_SIZE + 1][3];

void init_simlib(void);
void timing(void);
void schedule(double, int);
double expon(double, int);

#endif


Writing simlib.h


# Este es el simlib.c tambien necesario para correr cdigo en SIMLIB

In [ ]:
# Crear simlib.c
%%writefile simlib.c
#include "simlib.h"

int next_event_type, maxatr;
double sim_time, event_list[EVENT_LIST_SIZE + 1][3];

void init_simlib(void) {
    sim_time = 0.0;
    for (int i = 1; i <= EVENT_LIST_SIZE; ++i) {
        event_list[i][EVENT_TIME] = INFINITY;
        event_list[i][EVENT_TYPE] = 0;
    }
}

void timing(void) {
    int i;
    double min_time = INFINITY;

    next_event_type = 0;

    for (i = 1; i <= EVENT_LIST_SIZE; ++i) {
        if (event_list[i][EVENT_TIME] < min_time) {
            min_time = event_list[i][EVENT_TIME];
            next_event_type = i;
        }
    }

    if (next_event_type == 0) {
        printf("La lista de eventos está vacía en el tiempo %.2f\n", sim_time);
        exit(1);
    }

    sim_time = event_list[next_event_type][EVENT_TIME];
    event_list[next_event_type][EVENT_TIME] = INFINITY;
}

void schedule(double time, int type) {
    event_list[type][EVENT_TIME] = time;
    event_list[type][EVENT_TYPE] = type;
}

double expon(double mean, int stream) {
    double u = rand() / (RAND_MAX + 1.0);
    return -mean * log(1.0 - u);
}


Writing simlib.c


# **Ejercicio 2.3**

Un promedio de 10 personas/hora con distribución de Poisson entran a los carriles centrales de una piscina con el fin de nadar un rato. En promedio, cada persona nada 30 minutos con distribución exponencial. En la piscina existen tres carriles disponibles para este tipo de nadadores. Si un nadador se encuentra sólo en un carril, nadara pegado siempre al lado derecho del carril; en caso de que otra persona entre en ese carril cada nadador realizara su ejercicio por un extremo del carril. Debido a políticas de seguridad nunca puede haber más de dos nadadores en un mismo carril, por esto, si un nadador llega y están ocupados los carriles se retira enojado y no regresa.
¿Cuál es la proporción del tiempo en la que habrá tres personas nadando?
¿Cuántas personas en promedio se encuentran nadando en la piscina?
¿Cuántos carriles es necesario asignar para asegurar que el 95% de este tipo de nadadores que llegan a la piscina puedan entrenar?


# **Desarrollo en SimPy**

In [ ]:
import simpy
import random

# Parámetros del sistema
TIEMPO_ENTRE_LLEGADAS = 6      # minutos, distribución exponencial (10 personas/hora)
TIEMPO_NADO_PROMEDIO = 30      # minutos, distribución exponencial
NUM_CARRILES = 3
CAPACIDAD_CARRIL = 2
TIEMPO_SIMULACION = 10000      # minutos

MAX_NADADORES = NUM_CARRILES * CAPACIDAD_CARRIL

# Variables globales para estadísticas
estadisticas_estado = [0.0] * (MAX_NADADORES + 1)
estado_actual = 0
tiempo_anterior = 0

total_llegadas = 0
total_rechazados = 0
nadadores_satisfechos = 0


def actualizar_estado(env):
    global tiempo_anterior
    estadisticas_estado[estado_actual] += env.now - tiempo_anterior
    tiempo_anterior = env.now


def nadador(env, piscina, id):
    global estado_actual, nadadores_satisfechos, total_rechazados

    with piscina.request() as req:
        resultado = yield req | env.timeout(0)  # Intenta tomar recurso sin esperar
        if req in resultado:
            # Entró a nadar
            estado_actual += 1
            actualizar_estado(env)
            nadadores_satisfechos += 1
            tiempo_nado = random.expovariate(1.0 / TIEMPO_NADO_PROMEDIO)
            yield env.timeout(tiempo_nado)
            estado_actual -= 1
            actualizar_estado(env)
        else:
            # Rechazado porque no hay espacio
            total_rechazados += 1


def generador_nadadores(env, piscina):
    global total_llegadas
    while True:
        yield env.timeout(random.expovariate(1.0 / TIEMPO_ENTRE_LLEGADAS))
        total_llegadas += 1
        env.process(nadador(env, piscina, total_llegadas))


def simular():
    global estado_actual, tiempo_anterior, estadisticas_estado, total_llegadas, total_rechazados, nadadores_satisfechos
    estado_actual = 0
    tiempo_anterior = 0
    total_llegadas = 0
    total_rechazados = 0
    nadadores_satisfechos = 0
    estadisticas_estado = [0.0] * (MAX_NADADORES + 1)

    env = simpy.Environment()
    piscina = simpy.Resource(env, capacity=MAX_NADADORES)
    env.process(generador_nadadores(env, piscina))
    env.run(until=TIEMPO_SIMULACION)

    total_tiempo = sum(estadisticas_estado)
    promedio_nadadores = sum(i * (estadisticas_estado[i] / total_tiempo) for i in range(MAX_NADADORES + 1))
    proporcion_tres_nadadores = estadisticas_estado[3] / total_tiempo
    porcentaje_rechazados = 100 * total_rechazados / total_llegadas if total_llegadas > 0 else 0

    print(f"Promedio de nadadores en la piscina: {promedio_nadadores:.2f}")
    print(f"Proporción del tiempo con 3 nadadores: {proporcion_tres_nadadores:.4f}")
    print(f"Porcentaje de nadadores rechazados: {porcentaje_rechazados:.2f}%")


simular()


Promedio de nadadores en la piscina: 3.90
Proporción del tiempo con 3 nadadores: 0.1924
Porcentaje de nadadores rechazados: 18.35%


# **Desarrollo en SIMLIB**

In [ ]:
%%writefile simulacion1.c
#include <stdio.h>
#include <stdlib.h>
#include "simlib.h"

#define LLEGADA 1
#define SALIDA 2
#define FIN_SIMULACION 3

#define TIEMPO_NADO 30.0
#define TASA_LLEGADAS 10.0
#define NUM_CARRILES 3
#define CAPACIDAD_TOTAL (NUM_CARRILES * 2)
#define TIEMPO_TOTAL 10000.0

int total_llegadas = 0, rechazados = 0;
double tiempo_estado[CAPACIDAD_TOTAL + 1];
int estado_actual = 0;

void llegada(void);
void salida(void);
void actualizar_estadisticas(void);

int main() {
    init_simlib();
    maxatr = 2;
    sim_time = 0.0;

    // Inicializa vector de tiempo en estado
    for (int i = 0; i <= CAPACIDAD_TOTAL; i++)
        tiempo_estado[i] = 0.0;

    // Programa la primera llegada y el fin de simulación
    schedule(sim_time + expon(60.0 / TASA_LLEGADAS, 1), LLEGADA);
    schedule(TIEMPO_TOTAL, FIN_SIMULACION);

    while (1) {
        timing();

        switch (next_event_type) {
            case LLEGADA:
                llegada();
                break;
            case SALIDA:
                salida();
                break;
            case FIN_SIMULACION:
                actualizar_estadisticas();

                printf("\n--- Resultados de la simulación ---\n");
                double tiempo_total = sim_time;
                double promedio = 0.0;

                for (int i = 0; i <= CAPACIDAD_TOTAL; i++) {
                    double propor = tiempo_estado[i] / tiempo_total;
                    if (i == 3)
                        printf("Proporción del tiempo con 3 nadadores: %.4f\n", propor);
                    promedio += i * propor;
                }

                printf("Promedio de nadadores en la piscina: %.2f\n", promedio);
                printf("Porcentaje de nadadores rechazados: %.2f%%\n",
                       100.0 * rechazados / total_llegadas);
                return 0;
        }
    }
}

void llegada(void) {
    actualizar_estadisticas();
    total_llegadas++;

    if (estado_actual < CAPACIDAD_TOTAL) {
        estado_actual++;
        schedule(sim_time + expon(TIEMPO_NADO, 2), SALIDA);
    } else {
        rechazados++;
    }

    schedule(sim_time + expon(60.0 / TASA_LLEGADAS, 1), LLEGADA);
}

void salida(void) {
    actualizar_estadisticas();
    if (estado_actual > 0)
        estado_actual--;
}

void actualizar_estadisticas(void) {
    static double tiempo_anterior = 0.0;
    double tiempo_actual = sim_time;
    tiempo_estado[estado_actual] += tiempo_actual - tiempo_anterior;
    tiempo_anterior = tiempo_actual;
}




Writing simulacion1.c


# **Para compilar y ejecutar el codigo de SIMLIB**

In [ ]:
!gcc simulacion1.c simlib.c -o simulacion -lm
!./simulacion


--- Resultados de la simulación ---
Proporción del tiempo con 3 nadadores: 0.0002
Promedio de nadadores en la piscina: 5.81
Porcentaje de nadadores rechazados: 81.87%
